# Applied AI Lab WS2022 Introductory Task

Please solve the following task as part of your application to the Applied AI Lab.
Since the lab depends on the participants being able to implement (deep) machine learning solutions, solving this simple introductory task is a requirement for acceptance.
However, it is not important to maximize the performance or do excessive optimization.

You are given a dataset (in Studip) of handwritten numbers with two digits.
Your task is to construct and train a classifier to predict the number from the image input.
Specifically, please:
1. Train an sklearn classifier for this task.
2. Train a deep learning model for this task with testing accuracy over 80% (there is no hidden test set, so do not overfit on the given test data).
3. Send a link to your solution (a single Google Colab notebook) as part of your application mail. A copy of this notebook is available at https://colab.research.google.com/drive/1MrL2dlC5c29sRcZDw3P8rk4KmyKpWd0U?usp=sharing.

Additional information:
* The dataset can be easily loaded with `torchvision.datasets.ImageFolder`.
* You can use any deep learning framework you are comfortable with.
* We recommend working on Google Colab. To have access to the dataset, see https://colab.research.google.com/notebooks/io.ipynb.



In [63]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data
import torch.nn.functional as F
import torchvision
from torchvision import transforms
from PIL import Image

In [64]:
def check_image(path):
    try:
        im = Image.open(path)
        return True
    except:
        return False

In [65]:
img_size = 32

In [66]:
img_transforms = transforms.Compose([
    transforms.Resize((img_size,img_size)),
    #transforms.Normalize((0.5,), (0.5,)),
    transforms.Grayscale(num_output_channels=1),
    transforms.ToTensor(),
    ])

In [67]:
train_data_path = "./images/train"
train_data = torchvision.datasets.ImageFolder(root=train_data_path,transform=img_transforms, is_valid_file=check_image)

In [68]:
test_data_path = "./images/test"
test_data = torchvision.datasets.ImageFolder(root=test_data_path,transform=img_transforms, is_valid_file=check_image)

In [69]:
batch_size=100

In [70]:
train_data_loader = torch.utils.data.DataLoader(train_data, batch_size, shuffle=True)
test_data_loader  = torch.utils.data.DataLoader(test_data, batch_size)

In [71]:
class SimpleNet(nn.Module):
    def __init__(self):
        super(SimpleNet, self).__init__()
        self.fc1 = nn.Linear(1024, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128,100)

    def forward(self, x):
        x = x.view(-1, img_size*img_size)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        res=torch.Tensor()
        for i in range(len(x)):
            try:
                res = torch.cat((res, torch.argmax(x[i])), dim=1)
            except:
                res = torch.argmax(x[i])
        return x

In [72]:
simplenet = SimpleNet()

In [73]:
optimizer = optim.Adam(simplenet.parameters(), lr=0.005)

In [74]:
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

simplenet.to(device)

SimpleNet(
  (fc1): Linear(in_features=1024, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=128, bias=True)
  (fc3): Linear(in_features=128, out_features=100, bias=True)
)

In [75]:
def train(model, optimizer, loss_fn, train_loader, val_loader, epochs=20, device="cpu"):
    for epoch in range(epochs):
        training_loss = 0.0
        model.train()
        for batch in train_loader:
            optimizer.zero_grad()
            inputs, targets = batch
            inputs = inputs.to(device)
            targets = targets.to(device)
            output = model(inputs)
            loss = loss_fn(output,targets)
            loss.backward()
            optimizer.step()
            training_loss += loss.data.item() * inputs.size(0)
        training_loss /= len(train_loader.dataset)

        model.eval()
        num_correct = 0
        num_examples = 0
        valid_loss = 0.0
        for batch in val_loader:
            inputs, targets = batch
            inputs = inputs.to(device)
            output = model(inputs)
            targets = targets.to(device)
            loss = loss_fn(output,targets)
            valid_loss += loss.data.item() * inputs.size(0)
            correct = torch.eq(torch.max(F.softmax(output), dim=1)[1], targets).view(-1)
            num_correct += torch.sum(correct).item()
            num_examples += correct.shape[0]

        valid_loss /= len(val_loader.dataset)

        print('Epoch: {}, Training Loss: {:.2f}, Validation Loss: {:.2f}, accuracy = {:.2f}'.format(epoch, training_loss,
        valid_loss, num_correct / num_examples))

In [76]:
train(simplenet, optimizer,torch.nn.CrossEntropyLoss(), train_data_loader,test_data_loader, epochs=9, device=device)

/var/folders/jc/xkd5blwd6hz99wjjbm45kwsw0000gn/T/ipykernel_58320/967564360.py:32: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  correct = torch.eq(torch.max(F.softmax(output), dim=1)[1], targets).view(-1)


Epoch: 0, Training Loss: 2.43, Validation Loss: 1.26, accuracy = 0.64
Epoch: 1, Training Loss: 1.09, Validation Loss: 0.91, accuracy = 0.74
Epoch: 2, Training Loss: 0.75, Validation Loss: 0.86, accuracy = 0.75
Epoch: 3, Training Loss: 0.55, Validation Loss: 0.82, accuracy = 0.77
Epoch: 4, Training Loss: 0.42, Validation Loss: 0.78, accuracy = 0.79
Epoch: 5, Training Loss: 0.33, Validation Loss: 0.80, accuracy = 0.78
Epoch: 6, Training Loss: 0.24, Validation Loss: 0.77, accuracy = 0.81
Epoch: 7, Training Loss: 0.19, Validation Loss: 0.80, accuracy = 0.80
Epoch: 8, Training Loss: 0.14, Validation Loss: 0.92, accuracy = 0.81
